# Pronova LLM Run Model #
## Use this notebook to do the following ##
- Run the current model on a query
- start a flask api server that accepts a query and will return a response

In [1]:
# Load require libraries
import os
from qdrant_client import QdrantClient
from qdrant_client.http import models
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables from .env file
load_dotenv()

True

### Setup Qdrant connection ###

In [2]:
# Get the Qdrant API key from the environment variable
Qdrant_api_key = os.getenv('Qdrant_API_KEY')
if not Qdrant_api_key:
    raise ValueError("No Qdrant API key found in environment variables")
Qdrant_url = os.getenv('Qdrant_URL')
if not Qdrant_url:
    raise ValueError("No Qdrant URL found in environment variables")


# Initialize Qdrant client
try:
    Qclient = QdrantClient(
        url= Qdrant_url,
        api_key=Qdrant_api_key
    )
    print("Successfully connected to Qdrant")
except Exception as e:
    print(f"Failed to connect to Qdrant: {e}")
    raise

Successfully connected to Qdrant


### Setup OpenAI connection ###

In [3]:
# Get the OpenAI API key from the environment variable
OpenAI_api_key = os.getenv('OPENAI_API_KEY')
if not OpenAI_api_key:
    raise ValueError("No OpenAI API key found in environment variables")

OpenAI.api_key = OpenAI_api_key

### Get an OpenAI embedding from a text segment (Function) ###

In [4]:
# Function to get the embedding of a text
def get_embedding(text):
    client = OpenAI()
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding

### Retrieve similar chunks from query (Function) ###

In [29]:
def retrieve_relevant_chunks(collection_name, query, top_k=5, threshold=0.8):
    query_embedding = get_embedding(query)
    
    search_result = Qclient.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k
    )

    filtered_results = [
        result for result in search_result if result.score >= threshold
    ]
    

    contexts = [result.payload["text"] for result in filtered_results]
    urls = [result.payload.get("url") for result in filtered_results]
    
    return contexts, urls


### Rank response source importance (Function) ###

In [6]:
from collections import Counter

def file_ratios(files):
    total_files = len(files)
    counts = Counter(files)
    return {file: count*100 / total_files for file, count in counts.items()}


# file_ratios(["a", "a", "b", "c"])
# {'a': 0.5, 'b': 0.25, 'c': 0.25}

### Markdown Print Function ###

In [7]:
def print_markdown(md_text):
    display(Markdown(md_text))

### Generate Response from Query (Function) ###

In [ ]:
import numpy as np

def generate_response(collection_name, query, all_query, all_context, all_responses, all_files):
    print("Generating response for query:", query)
    # generate context for new query
    context, files = retrieve_relevant_chunks(collection_name, query)
    
    if "URL not found" in files:
        print("URL not found in files, removing it")
        # remove the URL not found from the list of files
        files.remove("URL not found")    

    system_role = "You are a specialized assistant that only provides advice on dog-related veterinary care. If a user asks about any other animal or topic outside of dog health, politely decline to answer and remind them that you only provide information about dogs. You will always start by asking the user their dog's name, age, and breed if they didn't already provide it."
    # Combine retrieved chunks into a single string
    context_text = "\n".join(context)

    if files:
        # print("Adding new files: ", files)
        all_files.extend(files)
    # append query and context to the running lists
    all_query.append(query)
    all_context.append(context_text)

    # print("All files used now:", all_files)
    # create the messages object using all the queries and contexts
    messages = [{"role": "system", "content": system_role}]

    for i in range(len(all_query)):
        messages.append({"role": "system", "content": "Use this context to answer my following question: " + all_context[i]})
        messages.append({"role": "user", "content": all_query[i]})
        if i < len(all_responses):
            messages.append({"role": "system", "content": all_responses[i]})
    
    # print(messages)


    # Generate a response using GPT-4
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages
    )
    all_responses.append(completion.choices[0].message.content)
    return all_query, all_context, all_responses, all_files


In [31]:
import json

# to confirm that each file has a properly stored citation
def get_citation(filename):
    # files_list = ["../sources/sources_amva.json", " ../sources/sources_petMD_allergies.json", "/sources_petMD_behavior.json",  "/source.json", "/sources_petMD_care_healthy_living.json", "/sources_petMD_nutrition.json", "/sources_petMD_procedures.json", "/sources_petMD_symptoms.json", "/sources_petMD.json"]
    files_list = [f"../sources/{file}" for file in os.listdir("../sources") if file.endswith('.json')]
    # print(files_list)
    citation = {}
    
    for file in files_list:
        with open(file, 'r', encoding='utf-8') as f:
            sources = json.load(f)
            if filename in sources:
                citation = sources[filename]
                break
    
    url = citation.get('URL', 'URL not found')
    author = citation.get('Author', 'Author not found')
    date = citation.get('Date', 'Date not found')
    topic = citation.get('Topic', 'Topic not found')
    
    
    return topic, url, author, date


### Playground (use this to test querys in the notebook)

In [ ]:
# Qclient.get_collections()
collection_name = "FullModel"
query = "My dog Bamba, a 5yo golden, has been limping after short walks."
query_list, context_list, response_list, files_list = generate_response(collection_name, query, [], [], [], [])


print(response_list[0])
for url in files_list:
    print(url)


Generating response for query: My dog Bamba, a 5yo golden, has been limping after short walks.
Adding new files:  ['https://www.petmd.com/dog/symptoms/why-is-my-dog-limping', 'https://www.petmd.com/dog/symptoms/why-is-my-dog-limping', 'https://www.petmd.com/dog/breeds/miniature-poodle', 'https://www.petmd.com/dog/symptoms/why-is-my-dog-limping', 'https://www.petmd.com//dog/conditions/musculoskeletal/c_dg_arthritis_septic']
I'm sorry to hear that Bamba is limping. Limping in dogs can be caused by various issues, ranging from minor injuries to more serious conditions. Since Bamba has been experiencing limping after short walks, it would be a good idea to have him evaluated by your veterinarian as soon as possible. They can determine the underlying cause of the limping and recommend appropriate treatment.

In the meantime, you should avoid any strenuous activities for Bamba and keep him comfortable. It’s best not to give him any medications without consulting your vet first. Providing low

### Lightweight Flask Server (for Frontend API testing) ###

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

collection_name = "FullModel" # for eyals db
# collection_name = "LLM_V1"  # for other db


app = Flask(__name__)
CORS(app)  # This will enable CORS for all routes

@app.route('/query', methods=['POST'])
def query_llm():
    data = request.json
    new_query = data.get('new_query')
    queries = data.get('queries')
    contexts = data.get('contexts')
    responses = data.get('responses')
    
    # if not new_query or not queries or not contexts or not responses:
    #     return jsonify({'error': 'New query, queries, contexts, and responses must be provided'}), 400

    try:
        updated_queries, updated_contexts, updated_responses = generate_response(collection_name, new_query, queries, contexts, responses)
        return jsonify({
            'queries': updated_queries,
            'contexts': updated_contexts,
            'responses': updated_responses
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)